<a href="https://colab.research.google.com/github/megawattfs/schrayer-msai-research/blob/main/eye_movement_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Eye Movement Classifiation Notebook

Margaret Schrayer

University of Georgia Institute for Artificial Intelligence

In [ ]:
import numpy as np
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [ ]:
def computegazeEventClassification(gazeTargetMatrix,minPursuitDuration)
  '''
  %% This section initializes sampling frequency (Fs), 
  % the parameters for the Savitzky Golay Filter (Equation 4),
  % noise_threshold (Eq. 11, 12), Rotation Matrix (Eq. 2), H (Equation 2),
  % visual_angle (Eq. 9), and calibration_error (Eq. 9). 
  '''
  Fs=1000
  savitzky_golay_filter_parameter_f = 41
  savitzky_golay_filter_parameter_k = 8
  noise_threshold_eta = 0.2
  Rotation_Matrix = eye(3) #We assume a simple rotation matrix for Equation 1. TODO
  H=330;   #Specify this value in mm. 
  visual_angle = 3.0
  calibration_error = 0.5 #This is not a constant and will vary from subject to subject. 
  delta = visual_angle + calibration_error

  '''
  %% This section takes the input matrix, gazeTargetMatrix. 
  %Column 1: Gaze Timestamps.
  %Column 2: Gaze X Coordinates in the XYZ frame. 
  %Column 3: Gaze Y Coordinates in the XYZ frame.
  %Column 4: Target X Coordinates in the XYZ frame.
  %Column 5: Target Y Coordinates in the XYZ frame. 
  %Column 6: Target X Velocity in the XYZ frame.
  %Column 7: Target Y Velocity in the XYZ frame.
  '''
  Gaze_TimeStamp=gazeTargetMatrix(:,1);
  Gaze_X=gazeTargetMatrix(:,2);
  Gaze_Y=gazeTargetMatrix(:,3);

  Target_X=gazeTargetMatrix(:,4);
  Target_Y=gazeTargetMatrix(:,5);
  Target_Velocity_Euclidean=(bsxfun(@hypot, gazeTargetMatrix(:,6), gazeTargetMatrix(:,7)));
  Target_Velocity_Angular=computeTargetSphericalVelocity(Target_X,Target_Y,Fs,savitzky_golay_filter_parameter)


  '''
  This section computes the gaze angular velocity (Eq. 6a)
  '''
  Gaze_prime=repmat([0;0;H],1,length(Gaze_X))+Rotation_Matrix*[Gaze_X Gaze_Y zeros(length(Gaze_X),1)]';
  Gaze_prime=Gaze_prime';
  Gaze_X_prime=Gaze_prime(:,1);
  Gaze_Y_prime=Gaze_prime(:,2);
  Gaze_Z_prime=Gaze_prime(:,3);
  Gaze_X_prime_dot_nofilt=derivative(Gaze_X_prime)./derivative(Gaze_TimeStamp);
  Gaze_X_prime_dot=sgolayfilt(Gaze_X_prime_dot_nofilt,savitzky_golay_filter_parameter.k,savitzky_golay_filter_parameter.f);
  Gaze_Y_prime_dot_nofilt=derivative(Gaze_Y_prime)./derivative(Gaze_TimeStamp);
  Gaze_Y_prime_dot=sgolayfilt(Gaze_Y_prime_dot_nofilt,savitzky_golay_filter_parameter.k,savitzky_golay_filter_parameter.f);
  Gaze_Z_prime_dot=0;
  Gaze_Abs_dot_prime=sqrt(Gaze_X_prime_dot.^2+Gaze_Y_prime_dot.^2+Gaze_Z_prime_dot.^2);
  [theta,phi,rho] = cart2sph(Gaze_X_prime, Gaze_Y_prime,Gaze_Z_prime);
    
  rho_dot=((Gaze_X_prime.*Gaze_X_prime_dot)+(Gaze_Y_prime.*Gaze_Y_prime_dot)+(Gaze_Z_prime.*Gaze_Z_prime_dot))./(rho);
  denominator_theta=(Gaze_X_prime.^2+Gaze_Y_prime.^2);
  numerator_theta=(Gaze_Y_prime.*Gaze_X_prime_dot)-(Gaze_X_prime.*Gaze_Y_prime_dot);
  theta_dot=(numerator_theta./denominator_theta).*cos(phi);
  
  numerator_phi=Gaze_Z_prime.*((Gaze_X_prime.*Gaze_X_prime_dot)+(Gaze_Y_prime.*Gaze_Y_prime_dot));   %The second term in the numerator is zero because Gaze_Z_prime_dot=0;
  denominator_phi=(rho.^2).*sqrt(denominator_theta);
  phi_dot= numerator_phi./denominator_phi;
  
  gazeVelocityVector=(bsxfun(@hypot, theta_dot, phi_dot))*(180/pi); %Convert to Degrees. 
  gazeAccVector_noFilt=derivative(gazeVelocityVector)*Fs;
  gazeAccelerationVector=sgolayfilt(gazeAccVector_noFilt,savitzky_golay_filter_parameter.k,savitzky_golay_filter_parameter.f);
    
  '''
  %% This section computes the Foveal Visual Radius (Eq. 9)
  '''
  FVR=rho*tand(delta*0.5).*csc(asin(H./rho)); 

  '''   
  %% This section computes the saccade velocity threshold (Eq. 10a)
  '''
  gazeVelocityThreshold=computegazeVelocityThreshold(gazeVelocityVector);
  
  '''
  %% This condition checks for the condition in Eq. 12 for smooth pursuits.  
  '''
      
  Tr=ones(length(Gaze_X_prime),1)*10; %Convert Tr to mm by multiplying it with 10. 
  Dist_X=Gaze_X_prime-Target_X;
  Dist_Y=Gaze_Y_prime-Target_Y;
  Abs_Dist_Gaze_Hand=(bsxfun(@hypot, Dist_X, Dist_Y));
    
  gazeEventVector=computeGazeEventVector(gazeVelocityVector,gazeVelocityThreshold,Fs,minPursuitDuration,gazeAccelerationVector);
  %This function returns a vector classifying all saccades as 1s and
  %everything else as zero. Fixations and smooth pursuits should be
  %searched in the vector elements that are '0'. This function implements
  %the checks in Equations 12 c-d. 
  
  gazeEventVector((Abs_Dist_Gaze_Hand<=(1+noise_threshold_eta)*(FVR+Tr)) & gazeEventVector==0)=2; %Assign gaze events to 2 (smooth pursuits) if Eq. 12a is satisifed. 
  gazeEventVector(abs(Target_Velocity_Angular-gazeVelocityVector)>(noise_threshold_eta*gazeVelocityVector))=0; % If 12b is not satisfied, then turn events back to unclassified. 
    
  
  [c,valnew] = accumconncomps(gazeEventVector)
  Z=  [c,valnew]
  smoothPursuitsVector=length(find(Z(:,1)==2 & Z(:,2)>=minPursuitDuration))

In [ ]:
'''
%% Function to compute the gazeVelocityThreshold
%One thing to note while using this function is that the input vector
%should comprise of multiple trials (discrete tasks) or as many time points
%as possible if the task is continuous. The more the data points, the
%better the estimate of the saccade velocity threshold.
'''

def computegazeVelocityThreshold(varargin)
  '''
  %This function computes and plots the parameters of a bimodal lognormal
  %plot. The input data for this function is just a vector. 
  % INPUTS:
  %Data_Vector      =       gaze velocity vector after blink correction and artifact removal. 
  %This value should be in degree/sec. 
  %Max_Iteration_MLE      =   maximum iterations for mle
  %Max_Fun_Evals_MLE      =     PDF function evaluation limit
  %pStart     =       default value 0.2. Shows the mixing ratio of the two pdfs.
  %We start with 0.2 unless otherwise specified. 
  %Data_Peak_Threshold    =   This threshold is required to determine a lower
  %bound for the findpeaks function. Any velocity peak below the Threshold
  %will be ignored. The default value for this is 0.05. 
  %Bin Size = Self-explanatory. 
  %Gaze_Fs = Sampling Frequency of the Gaze Data

  % OUTPUTS:
  %   Velocity_Threshold = Gaze Velocity Threshold based on the lognormal plot.
  %   ParamEsts     = Estimates of the parameters of the bimodal lognormal distribution.
  %   h     = Graphic handle for the bar plot.
  %   PDFGRID        = The fitted PDF function.


  %The easiest way to use the function is:
  %gazeVelocityThreshold=computegazeVelocityThreshold(Data_Vector);

  %More details about the implementation can be found here
  %(http://www.mathworks.com/help/stats/examples/fitting-custom-univariate-distributions.html)

  %Copyright: Tarkeshwar Singh 2015. Dept. of Exercise Science,USC, Columbia,SC.
  '''

  '''
  %% This section sets the initial parameters if they are not entered as inputs. 
  '''
  Max_Iteration_MLE = 600
  Max_Fun_Evals_MLE = 800
  pStart = 0.1
  muStart_Range = [.15 .85]
  Data_Peak_Threshold = 0.5;
  Bin_Size = .2;
  Gaze_Fs = 500;
  #%% This variable looks for local velocity peaks that are at least 50 ms apart.
  gaze_threshold_ts=50/(1000/(Gaze_Fs)); %Set it to 50 ms 
  
  '''
  %% This section takes the input parameters and assigns them to local variables. 
  '''
  if nargin < 1:
      error('myApp:argChk', 'Wrong number of input arguments')
  
  elif nargin == 1:
    Data_Vector=varargin{1};

  elif nargin == 2:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};

  elif nargin == 3:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};
    Max_Fun_Evals_MLE =varargin{3};

  elif nargin==4:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};
    Max_Fun_Evals_MLE =varargin{3};
    pStart=varargin{4};

  elif nargin==5:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};
    Max_Fun_Evals_MLE =varargin{3};
    pStart=varargin{4};
    muStart_Range = varargin{5};

  elif nargin==6:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};
    Max_Fun_Evals_MLE =varargin{3};
    pStart=varargin{4};
    muStart_Range = varargin{5};
    Data_Peak_Threshold=varargin{6};

  elif nargin==7:
    Data_Vector=varargin{1};
    Max_Iteration_MLE=varargin{2};
    Max_Fun_Evals_MLE =varargin{3};
    pStart=varargin{4};
    muStart_Range = varargin{5};
    Data_Peak_Threshold=varargin{6};
    Bin_Size=varargin{7};

  elif nargin>7:
    error('myApp:argChk', 'Wrong number of input arguments');

  [pks,locs] = findpeaks(Data_Vector,'minpeakheight',Data_Peak_Threshold,'minpeakdistance',gaze_threshold_ts);
  '''
  % This function computes local velocity peaks under two constraints: a)
  % minpeakheight (velocity should be at least 0.5 deg/s, default or user
  % specified value); and b) minpeakdistance (peaks should be separated a certain distance).
  % The local peaks are also log transformed here. 
  '''
  lengthDataVector=length(Data_Vector);
  Sorted_Peak_Vector=sort(pks);
  x=log(Sorted_Peak_Vector);  %Log transform of the local velocity peaks. 

  '''
  %% We now define the model and provide the intial guess for the parameters. 
  %The PDF for a mixture of two normals is just a weighted sum of the PDFs of 
  %the two normal components, weighted by the mixture probability. 
  '''
  pdf_normmixture = @(x,p,mu1,mu2,sigma1,sigma2) p*normpdf(x,mu1,sigma1) + (1-p)*normpdf(x,mu2,sigma2);
  muStart = quantile(x,muStart_Range);
  sigmaStart = sqrt(var(x) - .25*diff(muStart).^2);
  start = [pStart muStart sigmaStart sigmaStart];
  lb = [0 -Inf -Inf 0 0];
  ub = [1 Inf Inf Inf Inf];
  options = statset('MaxIter',Max_Iteration_MLE, 'MaxFunEvals',Max_Fun_Evals_MLE);
  paramEsts = mle(x, 'pdf',pdf_normmixture, 'start',start, 'lower',lb, 'upper',ub, 'options',options);

  '''
  %% This section creates a plot of the bimodal lognormal distribution
  '''    
  bins = 0:Bin_Size:max(x);
  figure('Color',[1 1 1]);
  h=bar(bins,histc(x,bins)/(length(x)*Bin_Size),'histc');
  set(h,'FaceColor',[.9 .9 .9],'linewidth',2);
  xgrid = linspace(1.1*min(x),1.1*max(x),200);
  pdfgrid = pdf_normmixture(xgrid,paramEsts(1),paramEsts(2),paramEsts(3),paramEsts(4),paramEsts(5));
  hold on; plot(xgrid,pdfgrid,'LineWidth',5, 'LineStyle','-', 'Color','m'); hold off
  xlabel('Ln(Velocity) of Local Peaks', 'fontsize',24,'fontweight','b','color','k'); 
  ylabel('Probability Density Function','fontsize',24,'fontweight','b','color','k');
  set(gca,'FontSize',24);
  set(gca, 'box', 'off');
  axis tight
  
  '''
  %% This section creates the output variables. 
  '''
  varargout{4}=pdfgrid;
  paramEsts(6)=(paramEsts(3)-paramEsts(2))/(2*(paramEsts(4)+paramEsts(5)));
  varargout{2}=paramEsts;
  varargout{3}=h;

  lower_saccade_vel=paramEsts(3)-(2*paramEsts(5));
  upper_fixation_vel=paramEsts(2)+(2*paramEsts(4));
  Velocity_Threshold=exp(0.5*((lower_saccade_vel)+(upper_fixation_vel)));

  #Note that if the distribution is not bimodal, the threshold would be biased. 

  varargout{1}=Velocity_Threshold;
  h2=vline(log(Velocity_Threshold),'g');
  set(gca,'LineWidth',2);
  set(h2,'LineWidth',2);
  text(log(Velocity_Threshold)+0.1,0.4,strcat(num2str(round(Velocity_Threshold)),'^{\circ}','/s'),'fontsize',24,'fontname','Helvetica') 

  '''
  %% This section plots a sub section of the gaze velocity data along with the 
  % velocity threshold computed using our method and the 30 deg/sec value
  % reported in the literature.
  '''
  figure('Color',[1 1 1]);
  PeakSig=Data_Vector((lengthDataVector/4+1):(lengthDataVector/2));
  time_vector=1:1:length(PeakSig);
  [pks,locs] = findpeaks(PeakSig,'minpeakheight',Velocity_Threshold,'minpeakdistance',25);
  h1=plot(time_vector,PeakSig,'linewidth',3);
  hold on
  h2=plot(time_vector(locs),pks+0.05,'k^','markerfacecolor',[1 0 0]);
  hold off
  xlabel('Time (ms)', 'fontsize',28,'fontweight','b','color','k'); 
  ylabel('Velocity (^o/s)','fontsize',28,'fontweight','b','color','k');
  set(gca,'FontSize',24);
  set(gca, 'box', 'off');
  set(h2,'markersize',12)
  axis tight
  h3=hline(Velocity_Threshold,'g');
  set(gca,'LineWidth',2);
  set(h3,'LineWidth',6);
  h4=hline(20,'c');
  set(gca,'LineWidth',2);
  set(h4,'LineWidth',6);

  return varargout

In [ ]:
'''
%% Function to compute the gaveEventVector
%This function creates an event vector for saccades. All time points that
%qualify as saccades are marked as '1' and everything else is left
%unassigned as '0'. 
'''

def computeGazeEventVector(gazeVelocityVector,saccadeVelocityThreshold,Fs,minFixationDuration,gazeAccelerationVector):
  '''
  %This function searches for inflection points on either sides of a saccade
  %and also verifies that the acceleration leading up to the velocity peak
  %exceeded the acceleration threshold of 6000 deg/sec^2. 

  % INPUTS:
  %gazeVelocityVector: gaze velocity vector. 
  %saccadeVelocityThreshold: Obtained from the function
  %computegazeVelocityThreshold.
  %Fs : Sampling Frequency.
  %minFixationDuration : We have set it at 40 ms. 
  %gazeAccelerationVector: gaze acceleration vector

  % OUTPUTS:
  %  onsetVelocityThreshold: The gaze velocity at which a saccade begins. 
  %  offsetVelocityThreshold: The gaze velocity at which a saccade
  %  terminates. 
  % gazeEventVector: outputs 1 for time points that qualify as saccade and 0
  % otherwise.

  %The easiest way to use the function is:
  %gazeEventVector=computeGazeEventVector(gazeVelocityVector,saccadeVelocityThreshold,Fs,minFixationDuration,gazeAccelerationVector);

  %Copyright: Tarkeshwar Singh 2015. Dept. of Exercise Science,USC, Columbia,SC.
  '''

  #%% Saccade Acceleration Threshold
  saccadeAccThreshold=6000

  '''
  %% This section plots the gaze anglualr velocity and also the saccade velocity threshold. 
  ''' 
  PeakSig=gazeVelocityVector
  x = list(range(1,length(PeakSig) + 1))
  locs, properties = find_peaks(PeakSig, height=saccadeVelocityThreshold, distance=(minFixationDuration*Fs/1000)) 
  #50 ms because we assume min fixation or sp is ~40 ms (6 points) and min. saccade is 20 (4 points)
  pks = properties['peak_heights']

  plt.plot(x,PeakSig), hold on
  plt.plot(x[locs],pks+0.05)
  plt.axhline(saccadeVelocityThreshold, label="Saccade Detection Threshold")
  plt.xlabel('Time Points (multiples of 5 ms)')
  plt.ylabel('Gaze Angular Velocity')
  plt.show()

  '''
  %% In this section, we search for inflection points within a 120 time point 
  %window. Since we sampled at 1000 Hz, 120 time points is reasonable but at
  %lower frequences this value may need to be changed. 
  '''
  locs_less_120=locs-(120*Fs/1000)
  locs_plus_120=locs+(120*Fs/1000)
  locs_for_onset_offset_search=[locs_less_120 locs locs_plus_120]
  locs_for_onset_offset_search(1:3,1) = max(1,locs_for_onset_offset_search(1:3,1))
  locs_for_onset_offset_search(end-2:end,3) = min(length(gazeVelocityVector),locs_for_onset_offset_search(end-2:end,3))
  gazeEventVector=zeros(length(gazeVelocityVector),1)
  gazeEventVector(gazeEventVector==0 & gazeAccelerationVector>=saccadeAccThreshold)=1 #Acceleration Threshold is implemented first. 

  for i=1:size(locs_for_onset_offset_search,1):
      gaze_velocity_onset_sector=gazeVelocityVector(locs_for_onset_offset_search(i,1):locs_for_onset_offset_search(i,2))
      onset_sector=derivative(gaze_velocity_onset_sector)
      
      if ~isempty(find(onset_sector<0,1,'last')):
        onsetVelocity(i)=gaze_velocity_onset_sector(find(onset_sector<0,1,'last')+1)
        onset_loc=find(onset_sector<0,1,'last')+locs_for_onset_offset_search(i,1)
      else:
        onsetVelocity(i)=8 #approx 8 deg/s is a fair assumption. 
        onset_loc=locs_for_onset_offset_search(i,1)
      
      gaze_velocity_offset_sector=gazeVelocityVector(locs_for_onset_offset_search(i,2):locs_for_onset_offset_search(i,3))
      offset_sector=derivative(gaze_velocity_offset_sector)
      
      if ~isempty(find(offset_sector>0,1,'first')) && (length(gaze_velocity_offset_sector)>=(find(offset_sector>0,1,'first')+1)):
        offsetVelocity(i)=gaze_velocity_offset_sector(find(offset_sector>0,1,'first')+1);
        offset_loc=find(offset_sector>0,1,'first')+locs_for_onset_offset_search(i,2);
      else:
        offsetVelocity(i)=9; %9 because of glissades in our data set. 
        offset_loc=locs_for_onset_offset_search(i,3);
      
      gazeEventVector(onset_loc:offset_loc)=1

  '''
  %% This section computes the onsetVelocityThreshold and offsetVelocityThreshold
  %by taking the average of the computed initiation and termination values.
  '''
  onsetVelocityThreshold=mean(onsetVelocity);
  offsetVelocityThreshold=mean(offsetVelocity);
  plot(gazeEventVector,'r')
  varargout{1}=gazeEventVector;
  varargout{2}=onsetVelocityThreshold;
  varargout{3}=offsetVelocityThreshold;

  return varargout

In [ ]:
'''
%% This function computes the angular velocity of the targets. 
'''
def computeTargetSphericalVelocity(Target_X,Target_Y,Fs,sg_frame_length,sg_order):
  '''
  %This function computes T_dot_phi_theta (Eq. 12b).
  %Inputs:
  %Target_X: Vector of X coordinates of target.
  %Target_Y: Vector of Y coordinates of target.
  %Fs: Sampling Frequency. 
  %savitzky_golay_filter_parameter: Filter parameters. 

  %Output:
  %TargetVelocityVector: Target Velocity in Spherical Coordinates.
  '''

  Rotation_Matrix = np.eye(3) #We assume a simple rotation matrix for Equation 1.
  h = 330 #Specify this value in mm.

  Target_prime= np.matmul(Rotation_Matrix,np.array([Target_X, Target_Y, [0]*(length(Target_X))])) + np.array([0]*length(Target_X),[0]*length(Target_X),[h]*length(Target_X))
  Target_prime=Target_prime.T
  Target_X_prime=Target_prime[:,0]
  Target_Y_prime=Target_prime[:,1]
  Target_Z_prime=Target_prime[:,2]

  Target_X_prime_dot_nofilt=np.gradient(Target_X_prime)*Fs
  Target_X_prime_dot=savgol_filter(Target_X_prime_dot_nofilt,sg_frame_length,sg_order)
  Target_Y_prime_dot_nofilt=np.gradient(Target_Y_prime)*Fs
  Target_Y_prime_dot=savgol_filter(Target_Y_prime_dot_nofilt,sg_frame_length,sg_order)
  Target_Z_prime_dot=0
  Target_Abs_dot_prime=np.sqrt(np.square(Target_X_prime_dot) + np.square(Target_Y_prime_dot) + np.square(Target_Z_prime_dot))

  [theta,phi,rho] = cart2sph(Target_X_prime, Target_Y_prime,Target_Z_prime)

  rho_dot = np.divide((np.multiply(Target_X_prime,Target_X_prime_dot)+np.multiply(Target_Y_prime,Target_Y_prime_dot)+np.multiply(Target_Z_prime,Target_Z_prime_dot)), rho)
  denominator_theta = np.square(Target_X_prime)+np.square(Target_Y_prime)
  numerator_theta = np.multiply(Target_Y_prime,Target_X_prime_dot) - np.multiply(Target_X_prime,Target_Y_prime_dot)
  theta_dot = np.multiply(np.divide(numerator_theta,denominator_theta), np.cos(phi))
  numerator_phi = np.multiply(Target_Z_prime,(np.multiply(Target_X_prime,Target_X_prime_dot)+np.multiply(Target_Y_prime,Target_Y_prime_dot))) #The second term in the numerator is zero because Target_Z_prime_dot=0
  denominator_phi = np.multiply(np.square(rho), np.sqrt(denominator_theta))
  phi_dot= np.divide(numerator_phi, denominator_phi)

  TargetVelocityVector = np.hypot(theta_dot, phi_dot) * (180/np.pi) #Convert to Degrees. 
  return TargetVelocityVector

In [ ]:
def cart2sph(x,y,z):
    azimuth = np.arctan2(y,x)
    elevation = np.arctan2(z,np.sqrt(x**2 + y**2))
    r = np.sqrt(x**2 + y**2 + z**2)
    return azimuth, elevation, r

In [ ]:
def accumconncomps(components,values=-1,fun=sum):
  '''
  % Construct array with accumulation of connected components 
  % 
  %     [c,valnew] = accumconncomps(cc,val)
  %     [c,valnew] = accumconncomps(cc,val,fun)
  %     [c,valnew] = accumconncomps(cc)
  %
  % accumconncomps creates vectors by accumulating elements in val using
  % connected components in cc. Connected components are subsequent,
  % identical elements in the vector cc.
  %
  % The input vectors cc and val must have same size. The output array c 
  % contains the values of each connected component 
  % (e.g. cc=[1 1 2 2 2 1 2 3 3] returns c=[1 2 1 2 3]). valnew contains the
  % aggregated values in val in each connected component.
  %
  % fun is a function handle that determines the aggregation mode (default:
  % @sum). fun must be a function that takes a vector and returns a scalar 
  % (e.g. @mean, @var, @(x) max(x)-min(x)).
  %
  % accumconncomps(subs) is an equal expression to 
  % accumconncomps(subs,ones(size(subs)),@sum) and counts the number of
  % elements in each connected component.
  %
  % Example 1:
  % Sum the values in val according to the connected components in cc.
  %     cc  = [1 1 2 1 1 1 3 3 3 4 2 2];
  %     val = [2.3 1.2 5 3 2 5 3.2 4.5 2 2.2 1.2 2.2];
  %     [c,valnew] = accumconncomps(cc,val,@sum)
  %     c =       1     2     1     3     4     2
  %     valnew =  3.5   5.0   10.0  9.7   2.2   3.4
  %
  % Author: Wolfgang Schwanghart (w.schwanghart[at]unibas.ch)
  % Date: 10. Sept. 2008
  '''
  if values == -1:
    values = [1] * len(components)
  
  current_component = components[0]
  current_values = []
  accumulated_components = []
  accumulated_values = []
  for i in range(len(components)):
    if components[i] == current_component:
      current_values.append(values[i])
    else:#New component
      accumulated_components.append(current_component)
      accumulated_values.append(fun(current_values))
      current_component = components[i]
      current_values = [values[i]]
  #end of loop:
  accumulated_components.append(current_component)
  accumulated_values.append(fun(current_values))

  return [accumulated_components, accumulated_values]